In [ ]:
import pandas as pd
import os
import sys
import ray
import glob
import random
import webdataset as wds

In [ ]:
files = glob.glob("/work/arxiv-papers/pdf/*.pdf")
random.shuffle(files)
len(files)

In [ ]:
help(wds.ShardWriter)

In [ ]:
def upload(fname):
    print(f"uploading: {fname}")
    assert os.system(f"gsutil -m cp {fname} gs://ocro-arxiv/pdfs/") == 0
    os.remove(fname)

In [ ]:
writer = wds.ShardWriter("arxiv-pdfs-%06d.tar", maxsize=1e9, maxcount=100, post=upload)
for fname in files:
    base = fname.split("/")[-1]
    prefix, suffix = base.rsplit(".", 1)
    with open(fname, "rb") as stream:
        data = stream.read()
    assert suffix == "pdf"
    writer.write({
        "__key__": prefix,
        "pdf": data
    })
writer.close()
